In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, Activation, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras import regularizers, models, layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import optuna
import wandb
import gc

In [2]:
gpus = tf.config.list_physical_devices('GPU')

if gpus:
    print("TensorFlow is using the GPU \n", gpus)
else:
    print("No GPU detected.")
    
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

TensorFlow is using the GPU 
 [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
gc.collect()
tf.keras.backend.clear_session()

In [4]:
from wandb.integration.keras import WandbMetricsLogger

wandb.require("core")
wandb.login()

wandb: WARNING `wandb.require('core')` is a no-op as it is now the default behavior.
wandb: Currently logged in as: emmdaz (emmdaz-zzz) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [5]:
# Carga de los datos 

df = pd.read_csv("/Plant-Pathology-Classificator/plant-pathology-2020-/train.csv")
df.head()

,image_id,healthy,multiple_diseases,rust,scab
0,Train_0,0,0,0,1
1,Train_1,0,1,0,0
2,Train_2,1,0,0,0
3,Train_3,0,0,1,0
4,Train_4,1,0,0,0


In [6]:
import os

df["label"] = df[["healthy", "multiple_diseases", "rust", "scab"]].idxmax(axis=1)

df["filepath"] = df['image_id'].apply(lambda x: os.path.join("/Plant-Pathology-Classificator/plant-pathology-2020-/images", f'{x}.jpg'))

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_temp = train_test_split(df, test_size = 0.5, stratify = df["label"], random_state = 4)

X_test, X_val = train_test_split(X_temp, test_size = 0.4, stratify = X_temp["label"], random_state = 4)

print("Train size:", len(X_train))
print("Validation size:", len(X_val))
print("Test size:", len(X_test))

Train size: 910
Validation size: 365
Test size: 546


In [8]:
mini_train, _ = train_test_split(X_train, test_size = 0.6, stratify = X_train["label"], random_state = 4)

mini_val, _ = train_test_split(X_val, test_size = 0.2, stratify = X_val["label"], random_state = 4)

print("Small training size:", len(mini_train))
print("Small val size:", len(mini_val))

Small training size: 364
Small val size: 292


In [9]:
datagen_train = ImageDataGenerator(
    rotation_range = 0.1,
    brightness_range = (0.25,1.5),
    channel_shift_range = 20.0,
    fill_mode = "nearest",
    cval = 128,
    horizontal_flip = True,
    vertical_flip = True,
    rescale=1./255,
    dtype = "float32"
    )

datagen_test_and_val = ImageDataGenerator(rescale=1./255, dtype="float32")

In [10]:
train = datagen_train.flow_from_dataframe(
    dataframe = X_train,
    x_col = 'filepath',
    y_col = 'label',
    target_size = (128,128),
    batch_size = 32
)

test = datagen_test_and_val.flow_from_dataframe(
    dataframe = X_test,
    x_col='filepath',
    y_col='label',
    target_size = (128,128),
    batch_size = 32
)

val = datagen_test_and_val.flow_from_dataframe(
    dataframe = X_val,
    x_col = 'filepath',
    y_col = 'label',
    target_size = (128,128),
    batch_size = 32
)

mini_train = datagen_train.flow_from_dataframe(
    dataframe = mini_train,
    x_col = 'filepath',
    y_col = 'label',
    target_size = (128,128),
    batch_size = 32
)

mini_val = datagen_test_and_val.flow_from_dataframe(
    dataframe = mini_val,
    x_col = 'filepath',
    y_col = 'label',
    target_size = (128,128),
    batch_size = 32
)

Found 910 validated image filenames belonging to 4 classes.
Found 546 validated image filenames belonging to 4 classes.
Found 365 validated image filenames belonging to 4 classes.
Found 364 validated image filenames belonging to 4 classes.


NameError: name 'mini_test' is not defined

In [ ]:
# Función para los bloques residuales

def residual_block(x, kernel, kernel_size, activation, dropout, dropout_rate, regularizer, r_2):
        
    residual = x  
        
    if dropout == "y":
        # Camino "principal"
        x = layers.Conv2D(kernel, (kernel_size, kernel_size), padding = "same",
                              activation = activation)(x)
        x = layers.BatchNormalization()(x)
        
        # Capa intermedia Dropot
        x = layers.Dropout(dropout_rate)(x)
            
        # Capa lineal
        x = layers.Conv2D(kernel, (kernel_size, kernel_size), padding = "same")(x)
        x = layers.BatchNormalization()(x)
            
    else: 
        # Camino "principal"
        x = layers.Conv2D(kernel, (kernel_size, kernel_size), padding = "same",
                                  activation = activation, kernel_regularizer = regularizers.l2(r_2))(x)
        x = layers.BatchNormalization()(x)
                
        # Capa lineal
        x = layers.Conv2D(kernel, (kernel_size, kernel_size), padding = "same",
                                  kernel_regularizer = regularizers.l2(r_2))(x)
        x = layers.BatchNormalization()(x)
            
#     if x.shape[-1] != residual.shape[-1]:
#         residual = layers.Conv2D(filters = kernel, kernel_size = (1, 1),
#                                  strides=strides, padding = "same")(residual)
#         residual = layers.BatchNormalization()(residual)

    # Suma de la conexión residual
    x = layers.add([x, residual]) 
    x = layers.Activation(activation)(x)
        
    return x

In [ ]:
from tensorflow.keras import mixed_precision

mixed_precision.set_global_policy('mixed_float16')

In [ ]:
def objective(trial):

    tf.keras.backend.clear_session()

    model = models.Sequential()
    inputs = layers.Input(shape=(128, 128, 3))
    # x = data_augmentation(inputs, training = True)
    
    #############################################################################################################
    
    # Optuna sugiere función de activación para todas las capas
    activation = "relu"
    
    # Optuna sugiere regularizador
    regularizer = "L2"
    r_2 = trial.suggest_float("regularizer_value_2", 1e-6, 3e-6, log = True)
    
    # Optuna sugiere el número de capas
    n_layers = trial.suggest_int("N_layers", 10, 15)
    
    # Optuna sugiere número de kernels y su tamaño en la primer capa convolucional
    
    kernel_1 = 10
    size_1 = trial.suggest_categorical("Kernel_Size_1", [2,7])
    
    # Optuna sugiere Learning Rate y Optimizador
    
    lr = trial.suggest_float("learning_rate", 2.5e-4, 1e-3, log = True)
    
    optimizer_name = trial.suggest_categorical("optimizer", ["adam", "rmsprop"])
    
                              
    if optimizer_name == "adam":
        optimizer = tf.keras.optimizers.Adam(learning_rate = lr)
                              
    else:
        optimizer = tf.keras.optimizers.RMSprop(learning_rate = lr)
    
    #############################################################################################################
    
    # Primera convolución
    x = layers.Conv2D(kernel_1, (size_1,size_1), padding = "same")(inputs)
    x = layers.Activation(activation)(x)
    x = layers.BatchNormalization()(x)
    
    kernel_per_layer = [kernel_1]
    kernel_size_per_layer = [size_1]
    
    # Optuna sugiere número de kernels su tamaño y función de activación; también sugiere Dropout
    # y regularizadores
    
    dropout_per_layer = []
    dropout_percentage_per_layer = []
    
    n_kernel =  trial.suggest_int("Kernels", 12,16)
    
    kernel_size = trial.suggest_categorical(f"Kernel_Size", [3,5])
    kernel_size_per_layer.append(kernel_size)
    
    for i in range(n_layers):
                              
        dropout = trial.suggest_categorical(f"Dropout_L{i+2}", ["y", "n"])
        dropout_per_layer.append(dropout)
                              
        dropout_rate = trial.suggest_float(f"Dropout_value_L{i+2}",0.1, 0.2)
        
        # Capa Convolucional i-ésima:
        
        # Se elige entre Dropout o un Regularizador
        if dropout == "y":
            
            dropout_percentage_per_layer.append(dropout_rate)
            
            ker = int(n_kernel * (2 ** (i // 2)))
            kernel_per_layer.append(ker)
            
            x = layers.Conv2D(ker, (kernel_size, kernel_size), strides = 2, padding = "same",
                              activation = activation)(x)
            x = layers.BatchNormalization()(x)
            
            x = residual_block(x, ker, kernel_size, activation, dropout, dropout_rate, regularizer, r_2)
            
        else:
            dropout_percentage_per_layer.append(0.0)

            ker = int(n_kernel * (2 ** (i // 2)))
            kernel_per_layer.append(ker)

            x = layers.Conv2D(ker, (kernel_size, kernel_size), strides = 2, padding = "same",
                                  activation = activation, kernel_regularizer = regularizers.l2(r_2))(x)
            x = layers.BatchNormalization()(x)

            x = residual_block(x, ker, kernel_size, activation, dropout, dropout_rate, regularizer, r_2)
    
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.2)(x)
                              
    outputs = layers.Dense(4, activation = "softmax", dtype = "float32")(x)
        
    model = models.Model(inputs, outputs)
                              
    model.compile(
    optimizer = optimizer,
    loss = "categorical_crossentropy",
        metrics = ["accuracy"])
    
    #############################################################################################################

    wandb.init(
        project = "Plant-Pathology-Classificator-Conv2D-Residual-Trials-8.0",
        name = f"Trial_{trial.number}",
        reinit = True,
        config = {
            "kernel_1": kernel_1,
            "size_1": size_1,
            "activation": activation,
            "n_layers": n_layers,
            "kernel_per_layer": kernel_per_layer,
            "kernel_size_per_layer": kernel_size_per_layer,
            "regularizer": regularizer,
            "r_value2": r_2,
            "dropout_per_layer": dropout_per_layer,
            "dropout_percentage_per_layer": dropout_percentage_per_layer,
            "learning_rate": lr,
            "optimizer": optimizer_name,
        }
    )
    
    #############################################################################################################
                              
    early_stopping = EarlyStopping(monitor = 'val_accuracy', patience = 15, restore_best_weights = True)
    # lr_reduction = ReduceLROnPlateau(monitor='val_loss', factor = 0.5, patience = 5)
    
    #############################################################################################################
    """
    Creación del modelo
    """
    
    try:
        print(model.summary())
    
        history = model.fit(
            mini_train,
            validation_data = mini_val,
            epochs = 200,
            batch_size = 32,
            verbose = 1, 
            callbacks = [WandbMetricsLogger(log_freq = 5), early_stopping]
        )

        # val_loss = min(history.history["val_loss"])
        # train_loss = min(history.history["loss"])
        val_accuracy = max(history.history["val_accuracy"])
    
    except tf.errors.ResourceExhaustedError as e:
        print(f"Intento {trial.number} falló debido a: {e}")
        
        tf.keras.backend.clear_session()
        wandb.finish()
        gc.collect()
        
        return float("inf")

    except Exception as e:
        print(f"Intento {trial.number} falló. Unexpected error: {e}")
        
        tf.keras.backend.clear_session()
        wandb.finish()
        gc.collect()
        
        return float("inf")

    # Penalize overfitting
    
    # score = val_loss + 0.1 * (train_loss - val_loss)
    
    score = val_accuracy
    
    tf.keras.backend.clear_session()
    gc.collect()
    wandb.finish()

    return 1-score

In [ ]:
study = optuna.create_study(study_name = "Experimentos-Serie-8.0", direction = "minimize")
study.optimize(objective, n_trials = 200)

In [ ]:
print("Número de pruebas terminadas: ", len(study.trials))

trial = study.best_trial

print("Mejor intento: ", trial)

print("Valor: ", trial.value)
print("Hiperparámetros: ", trial.params)

In [ ]:
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_slice
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_rank

In [ ]:
import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [ ]:
plot_parallel_coordinate(study)

In [ ]:
plot_param_importances(study)